In [6]:
import sqlite3
from twitscrape.twitter_scraper import TwitterGeolocationScraper 
# twitscrape is a package I built and distributed as an aside learning experiment to this project. I plan to work on it further so the import method might change from the above.
# https://github.com/christywastaken/twitscrape 
# https://pypi.org/project/twitscrape/ 

newcastle_latitude = 54.975029
newcastle_longitude = -1.612477
search_radius_km = 15.0
summer_solstice_week_scraper = TwitterGeolocationScraper(start_date='2022-06-20', end_date='2022-06-28', latitude=newcastle_latitude, longitude=newcastle_longitude, radius=search_radius_km, filter_links=True, filter_replies=True, is_headless=True)
summer_solstice_week_df = summer_solstice_week_scraper.run()
# Creates dataframe of all the tweets in Newcastle-Upon-Tyne for a 15km radius the week around the summer solstice 2022. Links and replies filtered out. 



-- TwitterGeolocationScraper running. This may take a minute to update the webdriver. --
Page loading not complete
remaining rate limit: 249 | tweet_df length: 20
remaining rate limit: 248 | tweet_df length: 20
remaining rate limit: 247 | tweet_df length: 20
remaining rate limit: 246 | tweet_df length: 20
remaining rate limit: 245 | tweet_df length: 20
remaining rate limit: 244 | tweet_df length: 20
remaining rate limit: 243 | tweet_df length: 20
remaining rate limit: 242 | tweet_df length: 20
remaining rate limit: 241 | tweet_df length: 20
remaining rate limit: 240 | tweet_df length: 20
remaining rate limit: 239 | tweet_df length: 20
remaining rate limit: 238 | tweet_df length: 20
remaining rate limit: 237 | tweet_df length: 20
remaining rate limit: 236 | tweet_df length: 20
remaining rate limit: 235 | tweet_df length: 20
remaining rate limit: 234 | tweet_df length: 20
remaining rate limit: 233 | tweet_df length: 20
remaining rate limit: 232 | tweet_df length: 20
remaining rate limit:

In [5]:
#Store to SQL DB
conn = sqlite3.connect('twitter_sentiment_analysis.db')
summer_solstice_week_df.to_sql('summer_solstice_week', conn, if_exists='replace', index=False)
conn.close()

TypeError: can only join an iterable